In [1]:
%matplotlib notebook
import numpy as np
import scipy.linalg as la
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree

import scipy.sparse as sp
from scipy.sparse.linalg import spsolve, lsqr

from numpy.linalg import cond, norm, inv, lstsq

from scipy.optimize import minimize_scalar

from poly_basis import *
from spherepts import *
from rbf import *

from laplacebeltrami import *

from math import factorial as fac

In [2]:
n = 200
eps = None
k = 21
rbf_obj = rbf_dict['multiquadric']
# Choose solution and forcing fuction
solution_index = 10

nodes = gen_spiral_nodes(n)
normals = nodes

rbf = rbf_obj['rbf']
zeta  = rbf_obj['zeta']
d2rbf = rbf_obj['d2rbf']
Lrbf = lambda r,eps: 1*zeta(r,eps) + d2rbf(r,eps)

sol_deg = sphere_harm_degs[solution_index]
exact = lambda x: sphere_harm[solution_index](*x)*-sol_deg*(sol_deg+1)
forcing = lambda x: sphere_harm[solution_index](*x)
print('Harmonic degree: %d' % sol_deg)

Harmonic degree: 3


# Shakar-Wright Method

In [218]:
def grad_poly(nodes, projectors, deg):
    n = len(nodes)
    x = nodes[:,0]
    y = nodes[:,1]
    z = nodes[:,2]
    cols = fac(deg+3)//(fac(deg)*fac(3))
    P = np.zeros((n, cols))
    rhs_dx = np.zeros((cols, n))
    rhs_dy = np.zeros((cols, n))
    rhs_dz = np.zeros((cols, n))
    rhs_x = np.zeros((cols, n))
    rhs_y = np.zeros((cols, n))
    rhs_z = np.zeros((cols, n))
    i = 0
    for d in range(deg+1):
        #x^a y^b z^c with a+b+c = d
        for a in range(d,-1, -1):
            for b in range(d-a, -1, -1):
                c = d-a-b
                P[:,i] = x**a * y**b * z**c
                if a == 0:
                    rhs_dx[i] = 0
                else:
                    rhs_dx[i] = a * x**(a-1) * y**b * z**c
                if b == 0:
                    rhs_dy[i] = 0
                else:
                    rhs_dy[i] = b * x**a * y**(b-1) * z**c
                if c == 0:
                    rhs_dz[i] = 0
                else:
                    rhs_dz[i] = c * x**a * y**b * z**(c-1)
                i += 1
    for i, p in enumerate(projectors):
        rhs_x[:,i] = p[0,0]*rhs_dx[:,i] + p[1,0]*rhs_dy[:,i] + p[2,0]*rhs_dz[:,i]
        rhs_y[:,i] = p[0,1]*rhs_dx[:,i] + p[1,1]*rhs_dy[:,i] + p[2,1]*rhs_dz[:,i]
        rhs_z[:,i] = p[0,2]*rhs_dx[:,i] + p[1,2]*rhs_dy[:,i] + p[2,2]*rhs_dz[:,i]
    return P, rhs_x, rhs_y, rhs_z

def grad_rbf_outer(nodes, centers, zeta, epsilon):
    n_len = len(nodes)
    c_len = len(centers)
    r = dist_outer(nodes, centers)[:,:,np.newaxis]
    xs = (np.array(nodes).reshape((1,n_len,3)) - np.array(centers).reshape((c_len,1,3)))
    return zeta(r, epsilon) * xs

def SWM(nodes, normals, rbf_obj=rbf_dict['multiquadric'], epsilon=None, 
        stencil_size=15, poly_deg=None, poly_type='s'):
    n = len(nodes)
    k = stencil_size
    rbf = rbf_obj['rbf']
    zeta = rbf_obj['zeta']
    
    weights = np.zeros((n, stencil_size))
    row_index = [r for r in range(n) for c in range(stencil_size)]
    col_index = np.zeros((n, stencil_size))
    
    tree = cKDTree(np.array(nodes))
    projectors = [np.eye(3) - np.outer(node, node) for node in normals]
    
    for i, node in enumerate(nodes):
        stencil = tree.query(nodes[i], k)[1]
        col_index[i] = stencil
        nn = np.array([nodes[i] for i in stencil])
        nn_proj = np.array([projectors[i] for i in stencil])
        # center stencil
        nn -= nn[0]
        # scale stencil
        scale = np.max(np.abs(nn))
        nn /= scale        
        
        if poly_deg is None:
            P = None
        else:
            if poly_type == 'p':
                P, rhs_x, rhs_y, rhs_z = grad_poly(nn, nn_proj, poly_deg)
            elif poly_type == 's':
                P, rhs_x, rhs_y, rhs_z = gen_sphere_harm_basis(poly_deg, nn, nn_proj)
            rhs_x /= scale
            rhs_y /= scale
            rhs_z /= scale
            
        #return P
        
        dist_mat = dist_outer(nn,nn)
        #print(cond(rbf(dist_mat, 1)))
        if i==0 and epsilon is None and rbf_obj['shape']:
            if P is None or cond(P)>10**14:
                epsilon = optimize_eps(rbf, dist_mat, P=None, target_cond=10**12)
            else:
                epsilon = optimize_eps(rbf, dist_mat, P=P, target_cond=10**12)
            print('epsilon set: %g' % epsilon)
            
        A = rbf(dist_mat, epsilon)
        rhsAs = np.matmul(nn_proj, grad_rbf_outer(nn, nn, zeta, epsilon).reshape(
                (stencil_size,stencil_size,3,1))).reshape((stencil_size,stencil_size,3))
        rhsAs /= scale
        
        # For testing
        #return A, P, rhsAs[:,:,0], rhs_x, dist_mat
        
        if P is None:
            rhs = rhsAs[:,:,0] # only the x coordinates
            weights_grad = la.solve(A, rhs).T
            weights[i] = (weights_grad@weights_grad)[0]

            rhs = rhsAs[:,:,1] # only the y coordinates
            weights_grad = la.solve(A, rhs).T
            weights[i] += (weights_grad@weights_grad)[0]

            rhs = rhsAs[:,:,2] # only the z coordinates
            weights_grad = la.solve(A, rhs)[:stencil_size,:].T
            weights[i] += (weights_grad@weights_grad)[0]
        else:
            schur = True
            if schur:
                weights_grad = schur_solve(A, P, rhsAs[:,:,0], rhs_x)[0]
                weights[i] = (weights_grad@weights_grad)[0]
                weights_grad = schur_solve(A, P, rhsAs[:,:,1], rhs_y)[0]
                weights[i] += (weights_grad@weights_grad)[0]
                weights_grad = schur_solve(A, P, rhsAs[:,:,2], rhs_z)[0]
                weights[i] += (weights_grad@weights_grad)[0]
            else:
                num_basis = P.shape[1]

                AP = np.block([[A,P],[P.T, np.zeros((num_basis,num_basis))]])
                grad_rbf = []
                rhsA = rhsAs[:,:,0] # only the x coordinates
                rhs = np.block([[rhsA],
                               [rhs_x]])
                weights_grad = la.solve(AP, rhs)[:stencil_size,:].T
                weights[i] = (weights_grad@weights_grad)[0]

                rhsA = rhsAs[:,:,1] # only the y coordinates
                rhs = np.block([[rhsA],
                               [rhs_y]])
                weights_grad = la.solve(AP, rhs)[:stencil_size,:].T
                weights[i] += (weights_grad@weights_grad)[0]

                rhsA = rhsAs[:,:,2] # only the z coordinates
                rhs = np.block([[rhsA],
                                [rhs_z]])
                weights_grad = la.solve(AP, rhs)[:stencil_size,:].T
                weights[i] += (weights_grad@weights_grad)[0]

    C = sp.csc_matrix((weights.ravel(), (row_index, col_index.ravel())),shape=(n,n))
    return C

In [219]:
C = SWM(nodes, normals, rbf_obj, eps, 50, poly_deg=1, poly_type='p')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

epsilon set: 0.371308
216.37485816156186


In [149]:
A, P, f, g, dist_mat = SWM(nodes, normals, rbf_obj, None, 50, poly_deg=4, poly_type='p')

epsilon set: 0.370841


In [131]:
f = f[:,0]
g = g[:,0]

In [217]:
def schur_solve(A, P, f, g):
#     U, S, V = la.svd(P)
#     num = len(S)
#     S[S < 10**-10] = 0
#     P = (U[:,:num]*S)@V
    B = P.T @ la.solve(A, P)
    lam = lstsq(B, P.T@la.solve(A,f) - g, rcond=10**-14)[0]
    w = la.solve(A, f- P@lam)
    return w, lam

In [186]:
w, lam = schur_solve(A, P, f, g)
print(w)

[[ 6.70346747e-01  4.04951115e+00 -4.03302869e+00 ... -1.16451733e+01
   1.06297372e+00  1.50191262e+01]
 [-3.31307476e+00 -1.71518062e+00  2.05407473e+00 ...  4.49928747e+00
  -1.09062702e+01 -9.89172764e+00]
 [ 1.81454317e+00 -1.48044512e+00  2.62926304e-01 ...  7.96047283e+00
   2.46433465e+00  3.94957883e+00]
 ...
 [-1.04076400e-02 -2.49231895e-03 -1.71431034e-02 ...  1.40077106e+00
   1.28904486e-01 -2.72142669e-01]
 [ 5.10782882e-02  3.28633001e-02  2.26535043e-02 ...  8.12690541e-01
   5.66570513e+00 -2.15250935e-01]
 [-3.21170168e-03 -2.89114996e-02 -1.94910950e-02 ... -2.02139808e-01
   5.80393163e-01 -4.62289838e+00]]


In [187]:
AP = np.block([[A, P],[P.T, np.zeros((P.shape[1],P.shape[1]))]])
print(cond(AP))
c = np.block([[w],[lam]])
#c = np.block([w,lam])
rhs = np.block([[f],[g]])
#rhs = np.block([f,g])

2.3078437503726834e+19


In [188]:
print(np.allclose(AP @ c, rhs))
print(AP @ c - rhs)

True
[[ 8.30186996e-16 -1.17267307e-15  5.55111512e-17 ...  5.90499871e-15
   2.43555176e-14 -1.98452366e-15]
 [ 2.28983499e-16 -1.96911370e-15  1.38777878e-16 ... -1.48284163e-14
   1.50157664e-14  1.71251902e-14]
 [ 1.38777878e-17  1.17961196e-16  1.48406364e-15 ... -2.85362012e-15
   6.14092110e-15 -1.48769885e-14]
 ...
 [-3.45799593e-12  2.99984999e-12  1.34525296e-11 ...  1.01526773e-11
   5.45467838e-12  1.34996181e-11]
 [-1.22203829e-11 -1.18755545e-12 -4.31580469e-12 ...  3.00360015e-11
  -7.39919931e-12  5.05467890e-12]
 [-6.41390968e-11  3.34987771e-11 -6.70963333e-11 ...  3.24999472e-11
  -8.26936852e-11 -1.18099475e-10]]


In [142]:
P

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00, -1.53948759e-01, -3.52421726e-02, ...,
        -1.71404807e-02, -2.14838713e-04, -3.58701365e-03],
       [ 1.00000000e+00,  1.82312176e-01, -2.34947817e-02, ...,
        -1.00342935e-02,  2.75782445e-04, -5.65676033e-03],
       ...,
       [ 1.00000000e+00, -8.89346930e-01, -6.46106498e-01, ...,
        -1.36073209e+00,  4.26289719e-01,  2.38468633e-01],
       [ 1.00000000e+00, -2.29892248e-01, -6.10864325e-01, ...,
         1.32107616e+00, -4.07240487e-01,  4.87138319e-01],
       [ 1.00000000e+00,  7.52718847e-01, -5.05137808e-01, ...,
         1.76287233e-02, -4.30612228e-01, -1.33439519e+00]])

In [165]:
U, S, V = la.svd(P)

In [166]:
S[S < 10**-12] = 0
num = len(S)
(U[:,:num]*S)@V - P

array([[-3.10862447e-15,  4.53490640e-16,  6.29519122e-17, ...,
        -1.82471226e-16, -2.10335221e-17, -1.53739868e-16],
       [-6.66133815e-16,  1.94289029e-16,  1.02695630e-15, ...,
        -2.71355475e-17, -1.26199439e-17, -1.12085329e-16],
       [-9.99200722e-16,  8.32667268e-17, -7.21644966e-16, ...,
         8.04342487e-17,  4.60684279e-17,  1.47780250e-16],
       ...,
       [-1.88737914e-15, -6.66133815e-16, -1.11022302e-16, ...,
         0.00000000e+00, -2.49800181e-16, -2.77555756e-17],
       [-2.33146835e-15,  1.11022302e-16, -3.60822483e-16, ...,
         9.02056208e-17, -1.94289029e-16, -2.77555756e-17],
       [-1.77635684e-15,  7.77156117e-16, -6.66133815e-16, ...,
        -5.55111512e-17,  3.60822483e-16, -2.77555756e-17]])

In [154]:
S

array([8.33779500e+00, 4.97120460e+00, 4.66311974e+00, 3.10929536e+00,
       2.31851276e+00, 2.00862293e+00, 1.17478009e+00, 1.08497219e+00,
       8.88521909e-01, 7.62103780e-01, 5.24517637e-01, 4.69675151e-01,
       3.95559224e-01, 3.69598553e-01, 2.53684232e-01, 1.41083462e-01,
       1.31443373e-01, 8.79517335e-02, 7.96528630e-02, 5.83921321e-02,
       4.14072581e-02, 2.79865804e-02, 1.30832271e-02, 1.03078451e-02,
       4.42739297e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

# Tangent Plane Method

In [ ]:
C = TPM(nodes, normals, rbf_obj, eps, k)
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

In [ ]:
rbf_obj = rbf_dict['ninth degree PHS']
C = TPM(nodes, normals, rbf_obj, 1, k, poly_deg = 3)
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

In [ ]:
deg = 4
for a in range(deg,-1, -1):
    for b in range(deg-a, -1, -1):
        print(a, b, deg-a-b)

In [ ]:
from math import factorial as fac

In [ ]:
fac(4+3)/(fac(4)*fac(3))

In [ ]:
nodes = np.array([[7,2,3],[7,2,3]])

In [ ]:
P, rhs_x, rhs_y, rhs_z = grad_poly(nodes, 3)

In [ ]:
print(P)

In [ ]:
print(rhs_x)

In [ ]:
3*49

In [ ]:
node = nodes[0]

In [ ]:
node

In [ ]:
nn = nodes[:5]

In [ ]:
nn

In [ ]:
nn - node

In [ ]:
nn[1] - node